In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import imageio
import typing as tp

In [2]:
#constant

data_raw_frame = 100
opacity = 0.5

In [3]:
radar_df : pd.DataFrame = pd.read_csv(f"../data/processed data/radar_data_{0}.csv")
radar_df : pd.DataFrame = radar_df[radar_df["QPDH0"] < 0.3]

def change_withdelay(delta_t : float = 0.06) -> pd.DataFrame:
    """
    Moves radar_df to delta_t in time

    Parameters
    ----------
    delta_t : float
        Time travel radar_df
        

    Returns
    -------
    pd.DataFrame
        radar_df which has shifted to delta_t
    """

    with open(f"../data/raw data/radar_positions.json", "r") as file:
        radar_positions : dict[float, tp.Any] = {float(k): v for k, v in json.load(file).items()}

    for i, cords in radar_positions.items():
        for j, ax in enumerate(("X, (m)", "Y, (m)")):
            radar_df.loc[radar_df["radar_idx"] == i, ax] -= cords[j]

    vector_length : pd.Series = (radar_df['X, (m)']**2 + radar_df['Y, (m)']**2)**0.5
    radar_df['RadialDelta'] = (delta_t - radar_df['(radar_point_ts - lidar_ts), (s)']) * radar_df['AbsoluteRadialVelocity']
    radar_df['XwithDelta'] = radar_df['X, (m)'] * (vector_length + radar_df['RadialDelta']) / vector_length
    radar_df['YwithDelta'] = radar_df['Y, (m)'] * (vector_length + radar_df['RadialDelta']) / vector_length


    for i, cords in radar_positions.items():
        for j, ax in enumerate(("XwithDelta", "YwithDelta")):
            radar_df.loc[radar_df["radar_idx"] == i, ax] += cords[j]

    for i, cords in radar_positions.items():
        for j, ax in enumerate(("X, (m)", "Y, (m)")):
            radar_df.loc[radar_df["radar_idx"] == i, ax] += cords[j]
    
    return radar_df

In [ ]:

frame_num = 16
radar_df = pd.read_csv(f"../data/processed data/radar_data_{frame_num}.csv")
radar_df = radar_df[radar_df["QPDH0"] < 0.3]


for delta_time in tqdm(range(0, 300, 30)):
    frame_radar = pd.read_csv(f"../data/processed data/radar_data_{frame_num}.csv")
    frame_lidar = pd.read_csv(f"../data/processed data/lidar_data_{frame_num}.csv")

    plt.figure(figsize = (22, 10))
    plt.xlim([-110, 110])
    plt.ylim([-50, 50])
    
    frame_size = len(frame_lidar["r, (reflectance)"])

    gradient = []
    for i in range(frame_size):
        color = (frame_lidar["r, (reflectance)"][i] / 255) ** 0.3
        gradient.append((color, 0, 1 - color, opacity))
    
    plt.scatter(frame_lidar["X, (m)"], frame_lidar["Y, (m)"], s=1, c=gradient)
    plt.scatter(change_withdelay(delta_time/1000)["XwithDelta"], change_withdelay(delta_time/1000)["YwithDelta"], s=1, c = "red")

    plt.title(f"Frame {delta_time}")
    plt.savefig(f"../data/delta_t/Frame_{delta_time}.png")
    plt.close()


In [ ]:
with imageio.get_writer("../data/gif_delta_t.gif", mode="I", duration=0.5) as writer:
    for delta_time in range(0, 300, 30):
        image = imageio.imread(f"../data/delta_t/Frame_{delta_time}.png")
        writer.append_data(image)